## DataFrame Rationalization

We have data in two different formats. For large cities, the tables are organized as follows:

The primary columns are Date, City and Specie (pollutant). The remaining columns report the count, min, max, median and variance of the pollutant for that city on that particular date. 

`Unnamed: 0	Date	Country	City	Specie	count	min	max	median	variance
2	2979	06/04/2020	IN	Hyderabad	no2	95	0.1	31.2	6.1	848.08`

Thus, in order to extract either the pollutant or the City, we need to filter the rows: I use something along the lines of
`df[(df.City == c)]` and `df[(df.Specie == p)]` and these can ofcourse be combined. 

For rural areas, however, the data is organized differently. The primary columns are now the Data and the City, and for each, the remaining columns report the pollution levels for different pollutants.

`       	date	pm25	pm10	o3	no2	so2	co	city
3	06/01/2020	62	20	9	19	2	8	coimbatore`

So we can extract data for a city and then use the column to get the data for a pollutant, using a variant of `df[(df.City == c), 'pm25']`

This works fine when plotting things separately for rural and urban regions. However, what happens when we want to compare the data from the two tables? We need to convert these two datas into a single representation. Depending on what we want to do with the data, one format will be preferred over the other. In this sheet, I document how to do this both ways. 


In [3]:
import pandas as pd

----
First, let's load in the data and clean it up a bit:

In [12]:
year = 2020
df_r = pd.read_csv(f'data/{year}_7rural.csv')
df_u = pd.read_csv(f'data/{year}_7cities.csv')

In [13]:
print(df_r.head(5))
print(df_u.head(5))

         date  pm25  pm10  o3  no2  so2  co        city
0  03/01/2020    98    44  38   17    4   7  coimbatore
1  04/01/2020   102    27  26   30    3   5  coimbatore
2  05/01/2020    66    24  11   16    2  12  coimbatore
3  06/01/2020    62    20   9   19    2   8  coimbatore
4  07/01/2020    53    14   3   17    2   4  coimbatore
   Unnamed: 0        Date Country       City Specie  count  min    max  \
0        2977  28/02/2020      IN  Hyderabad    no2     96  3.0   32.7   
1        2978  09/03/2020      IN  Hyderabad    no2    120  0.3   27.1   
2        2979  06/04/2020      IN  Hyderabad    no2     95  0.1   31.2   
3        2980  11/04/2020      IN  Hyderabad    no2    113  0.2  500.0   
4        2981  08/05/2020      IN  Hyderabad    no2    120  0.8   23.1   

   median  variance  
0    12.2    405.29  
1     8.2    403.07  
2     6.1    848.08  
3     7.4  43973.00  
4     7.8    492.57  


In [44]:
# cleanup: rename the column titles for rural, remove min/max/count and variance for urban. 
df_r.rename(columns = {'date':'Date', 'city':'City'}, inplace=True)
df_u.drop(['Unnamed: 0', 'count', 'min', 'max', 'variance', 'Country'], axis=1, inplace=True)

# Fix the Dates for both and remove data that does not belong to the current year
df_r.Date = pd.to_datetime(df_r.Date, format = '%d/%m/%Y') 
df_u.Date = pd.to_datetime(df_u.Date, format = '%d/%m/%Y') 
    # we need the format string to specify which is the month and which is the year
df_r = df_r[df_r['Date'].dt.year == year]
df_u = df_u[df_u['Date'].dt.year == year]

KeyError: "['Unnamed: 0' 'count' 'min' 'max' 'variance' 'Country'] not found in axis"

In [15]:
print(df_r.head(5))
print(df_u.head(5))

        Date  pm25  pm10  o3  no2  so2  co        City
0 2020-01-03    98    44  38   17    4   7  coimbatore
1 2020-01-04   102    27  26   30    3   5  coimbatore
2 2020-01-05    66    24  11   16    2  12  coimbatore
3 2020-01-06    62    20   9   19    2   8  coimbatore
4 2020-01-07    53    14   3   17    2   4  coimbatore
        Date       City Specie  median
0 2020-02-28  Hyderabad    no2    12.2
1 2020-03-09  Hyderabad    no2     8.2
2 2020-04-06  Hyderabad    no2     6.1
3 2020-04-11  Hyderabad    no2     7.4
4 2020-05-08  Hyderabad    no2     7.8


----
### 1. Converting the Rural to the Urban format

The format of both tables is as above. For every city and date, we'd like to add an additional row for each pollutant. This is pretty easy to do using the `melt` method:

In [20]:
df_ru = pd.melt(df_r, id_vars=['City', 'Date'], value_vars=['pm25', 'pm10','o3', 'no2', 'so2', 'co'] )
df_ru.rename(columns = {'variable':'Specie', 'value':'Value'}, inplace=True)
df_uu = df_u.copy()
df_uu.rename(columns = {'median':'Value'}, inplace=True)
print("rural:", df_ru.shape)
print(df_ru.head(5))
print("urban:", df_uu.shape)
print(df_uu.head(5))

rural: (6240, 4)
         City       Date Specie  Value
0  coimbatore 2020-01-03   pm25     98
1  coimbatore 2020-01-04   pm25    102
2  coimbatore 2020-01-05   pm25     66
3  coimbatore 2020-01-06   pm25     62
4  coimbatore 2020-01-07   pm25     53
urban: (7601, 4)
        Date       City Specie  Value
0 2020-02-28  Hyderabad    no2   12.2
1 2020-03-09  Hyderabad    no2    8.2
2 2020-04-06  Hyderabad    no2    6.1
3 2020-04-11  Hyderabad    no2    7.4
4 2020-05-08  Hyderabad    no2    7.8


... and we're pretty much done! As a finishing touch, I'll just merge these into one table:

In [22]:
combo_u = pd.concat([df_uu, df_ru], sort=False)
combo_u

,Date,City,Specie,Value
0,2020-02-28,Hyderabad,no2,12.2
1,2020-03-09,Hyderabad,no2,8.2
2,2020-04-06,Hyderabad,no2,6.1
3,2020-04-11,Hyderabad,no2,7.4
4,2020-05-08,Hyderabad,no2,7.8
5,2020-01-03,Hyderabad,no2,10.0
6,2020-02-07,Hyderabad,no2,11.1
7,2020-03-11,Hyderabad,no2,10.4
8,2020-04-26,Hyderabad,no2,4.3
9,2020-05-19,Hyderabad,no2,10.2


----
### 2. Converting the Urban to the Rural format

In [23]:
print(df_r.head(5))
print(df_u.head(5))

        Date  pm25  pm10  o3  no2  so2  co        City
0 2020-01-03    98    44  38   17    4   7  coimbatore
1 2020-01-04   102    27  26   30    3   5  coimbatore
2 2020-01-05    66    24  11   16    2  12  coimbatore
3 2020-01-06    62    20   9   19    2   8  coimbatore
4 2020-01-07    53    14   3   17    2   4  coimbatore
        Date       City Specie  median
0 2020-02-28  Hyderabad    no2    12.2
1 2020-03-09  Hyderabad    no2     8.2
2 2020-04-06  Hyderabad    no2     6.1
3 2020-04-11  Hyderabad    no2     7.4
4 2020-05-08  Hyderabad    no2     7.8


What we want to do here is split up the urban data so that we have a column for each pollutant, and the primary columns are Dates and Citys

Fairly straightforward to accomplish:

In [37]:
# attempt 1:
df_ur = df_u.pivot_table(index=['City', 'Date'], columns='Specie')
df_ur
# df_ur.reset_index()

City       Date median                              
Specie                               co   no2    o3  pm10   pm25  so2
0           Bengaluru 2019-12-30    8.6  10.8  17.8  63.0  109.0  3.9
1           Bengaluru 2019-12-31    8.9  11.3  16.0  64.0  120.0  4.2
2           Bengaluru 2020-01-01    9.8  12.7  13.2  57.0   89.0  4.3
3           Bengaluru 2020-01-02    9.3  12.6  10.1  52.0   77.0  4.2
4           Bengaluru 2020-01-03    8.0  10.8  10.7  56.0   80.0  3.4
...               ...        ...    ...   ...   ...   ...    ...  ...
1273    Visakhapatnam 2020-06-25    3.3   7.7  12.3  40.0   42.0  3.9
1274    Visakhapatnam 2020-06-26    3.9  10.9   8.1  27.0   25.0  2.9
1275    Visakhapatnam 2020-06-27    4.4  11.6   8.8  50.0   59.0  4.0
1276    Visakhapatnam 2020-06-28    4.4  10.7  12.9  52.0   65.0  3.9
1277    Visakhapatnam 2020-06-29    4.9  17.9  10.5  66.0   85.0  5.9

[1278 rows x 8 columns]

In [40]:
# attempt 2
df_ur = df_u.set_index(['Date', 'City', 'Specie'])['median'].unstack().reset_index()

... and as before, merging with the rural data

In [43]:
combo_r = pd.concat([df_r, df_ur], sort=False)
combo_r

,Date,pm25,pm10,o3,no2,so2,co,City
0,2020-01-03,98.0,44.0,38.0,17.0,4.0,7.0,coimbatore
1,2020-01-04,102.0,27.0,26.0,30.0,3.0,5.0,coimbatore
2,2020-01-05,66.0,24.0,11.0,16.0,2.0,12.0,coimbatore
3,2020-01-06,62.0,20.0,9.0,19.0,2.0,8.0,coimbatore
4,2020-01-07,53.0,14.0,3.0,17.0,2.0,4.0,coimbatore
5,2020-01-13,143.0,51.0,31.0,30.0,4.0,2.0,coimbatore
6,2020-01-14,144.0,41.0,35.0,27.0,4.0,1.0,coimbatore
7,2020-01-15,124.0,31.0,31.0,28.0,4.0,1.0,coimbatore
8,2020-01-16,93.0,33.0,28.0,31.0,3.0,1.0,coimbatore
9,2020-01-17,94.0,33.0,18.0,25.0,4.0,1.0,coimbatore
